In [1]:
import torch
import pandas as pd
import astropy as ap
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
# from astropy.io import fits
import pdb
from scipy.ndimage.filters import maximum_filter1d
import glob
import fitsio as fits
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data_utils
from torch.nn.utils import clip_grad_norm_
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence, pad_sequence
from torch.distributions.normal import Normal
from tqdm import tqdm
from utils import collate_interp_sparse
import time

In [2]:
import sys
sys.path.insert(0, '../')
sys.path.insert(1, '../latent_ode/')
import latent_ode.lib as ode
import latent_ode.lib.utils as utils
from latent_ode.lib.latent_ode import LatentODE
from latent_ode.lib.ode_rnn import ODE_RNN
from latent_ode.lib.encoder_decoder import Encoder_z0_ODE_RNN, Decoder
from latent_ode.lib.diffeq_solver import DiffeqSolver
from latent_ode.lib.ode_func import ODEFunc

In [3]:
from ode_rnn import create_ODERNN_model

In [4]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")

In [5]:
# obsrv_std = torch.Tensor([0.1]).to(device)
# z0_prior = Normal(torch.Tensor([0.0]).to(device), torch.Tensor([1.]).to(device))
input_dim = 1
latent_dim = 40

In [6]:
model = create_ODERNN_model()

In [10]:
model_file = 'ode_rnn_state_tess.pth.tar'

In [11]:
state = torch.load(model_file, map_location=torch.device('cpu'))

In [12]:
model.load_state_dict(state['state_dict'])

RuntimeError: Error(s) in loading state_dict for ODE_RNN:
	Unexpected key(s) in state_dict: "ode_gru.z0_diffeq_solver.ode_func.gradient_net.6.weight", "z0_diffeq_solver.ode_func.gradient_net.6.weight". 
	size mismatch for ode_gru.GRU_update.update_gate.0.weight: copying a param with shape torch.Size([100, 84]) from checkpoint, the shape in current model is torch.Size([50, 82]).
	size mismatch for ode_gru.GRU_update.update_gate.2.weight: copying a param with shape torch.Size([40, 100]) from checkpoint, the shape in current model is torch.Size([40, 50]).
	size mismatch for ode_gru.GRU_update.reset_gate.0.weight: copying a param with shape torch.Size([100, 84]) from checkpoint, the shape in current model is torch.Size([50, 82]).
	size mismatch for ode_gru.GRU_update.reset_gate.2.weight: copying a param with shape torch.Size([40, 100]) from checkpoint, the shape in current model is torch.Size([40, 50]).
	size mismatch for ode_gru.GRU_update.new_state_net.0.weight: copying a param with shap

In [ ]:
model.eval()

### Input vec

In [ ]:
p = pd.read_pickle('pi_signal.pkl')

In [ ]:
p1, p2 = p

In [ ]:
p1

In [ ]:
ts = np.arange(0, 1, 1/len(p1))

In [ ]:
data = torch.FloatTensor((ts, p1))
data = data.unsqueeze(0)

In [ ]:
inp = collate_interp_sparse(data)

In [ ]:
observed = inp['observed_data'].to(device)
true = inp['data_to_predict'].to(device)
mask = inp['observed_mask'].to(device)
t = inp['observed_tp'].to(device)
x = torch.cat((observed, mask), -1)
# try:
#     z_u, z_std = model.encoder_z0.forward(x, t)
# except:
z_u, z_std = model.ode_gru.forward(x, t)

In [ ]:
x.shape